<a href="https://colab.research.google.com/github/Ignaciogg1/Challenger-cd_Alura1/blob/main/Copalurastorelatam_ipynb_txt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Importación de datos



In [ ]:
import pandas as pd
%pip install reportlab

url = "https://raw.githubusercontent.com/alura-es-cursos/challenge1-data-science-latam/refs/heads/main/base-de-datos-challenge1-latam/tienda_1%20.csv"
url2 = "https://raw.githubusercontent.com/alura-es-cursos/challenge1-data-science-latam/refs/heads/main/base-de-datos-challenge1-latam/tienda_2.csv"
url3 = "https://raw.githubusercontent.com/alura-es-cursos/challenge1-data-science-latam/refs/heads/main/base-de-datos-challenge1-latam/tienda_3.csv"
url4 = "https://raw.githubusercontent.com/alura-es-cursos/challenge1-data-science-latam/refs/heads/main/base-de-datos-challenge1-latam/tienda_4.csv"

tienda1 = pd.read_csv(url)
tienda2 = pd.read_csv(url2)
tienda3 = pd.read_csv(url3)
tienda4 = pd.read_csv(url4)

tienda2.head()

,Producto,Categoría del Producto,Precio,Costo de envío,Fecha de Compra,Vendedor,Lugar de Compra,Calificación,Método de pago,Cantidad de cuotas,lat,lon
0,Silla de oficina,Muebles,283600.0,15300.0,04/08/2021,Izabela de León,Medellín,1,Nequi,1,6.25184,-75.56359
1,Pandereta,Instrumentos musicales,76800.0,2300.0,26/11/2020,Blanca Ramirez,Medellín,3,Tarjeta de crédito,4,6.25184,-75.56359
2,Cama king,Muebles,443300.0,23800.0,22/10/2021,Izabela de León,Pereira,5,Tarjeta de crédito,10,4.81333,-75.69611
3,Cama king,Muebles,906200.0,46400.0,04/05/2022,Maria Alfonso,Cali,5,Tarjeta de crédito,10,3.43722,-76.52250
4,Silla de oficina,Muebles,397800.0,19400.0,26/02/2022,Lucas Olivera,Bogotá,4,Tarjeta de crédito,3,4.60971,-74.08175


#1. Análisis de facturación

1.1 Totales por tienda

In [ ]:
sumtienda1=tienda1['Precio'].sum()
sumtienda2=tienda2['Precio'].sum()
sumtienda3=tienda3['Precio'].sum()
sumtienda4=tienda4['Precio'].sum()

In [ ]:
print(f"Total Tienda 1 es:{sumtienda1:,.0f}")
print(f"Total Tienda 2 es:{sumtienda2:,.0f}")
print(f"Total Tienda 3 es:{sumtienda3:,.0f}")
print(f"Total Tienda 4 es:{sumtienda4:,.0f}")

Total Tienda 1 es:1,150,880,400
Total Tienda 2 es:1,116,343,500
Total Tienda 3 es:1,098,019,600
Total Tienda 4 es:1,038,375,700


1.2 Informes de ventas por año por tienda

In [ ]:
import pandas as pd
from tabulate import tabulate
from reportlab.lib.pagesizes import A4
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet

df = pd.DataFrame(tienda1)
df['Fecha de Compra'] = pd.to_datetime(df['Fecha de Compra'], dayfirst=True)
df['año'] = df['Fecha de Compra'].dt.year

resumen = df.groupby('año').agg(
    total_anual=('Precio', 'sum'),
    promedio_mensual=('Precio', lambda x: x.sum() / len(x))
).reset_index()

total_general = resumen['total_anual'].sum()
resumen['porcentaje'] = (resumen['total_anual'] / total_general) * 100

fila_total = pd.DataFrame({
    'año': ['TOTAL'],
    'total_anual': [total_general],
    'promedio_mensual': [resumen['promedio_mensual'].mean()],
    'porcentaje': [100]
})
resumen = pd.concat([resumen, fila_total], ignore_index=True)

resumen_pdf = resumen.copy()

resumen['total_anual'] = resumen['total_anual'].map('${:,.0f}'.format)
resumen['promedio_mensual'] = resumen['promedio_mensual'].map('${:,.0f}'.format)
resumen['porcentaje'] = resumen['porcentaje'].map('{:.2f}%'.format)

print("\n" + "="*60)
print(" INFORME DE VENTAS POR AÑO TIENDA 1".center(60, "="))
print("="*60)
print(tabulate(resumen, headers="keys", tablefmt="fancy_grid", showindex=False))
print("="*60)

pdf_file = "/content/informe_ventas_tienda1.pdf"
doc = SimpleDocTemplate(pdf_file, pagesize=A4)
elements = []
styles = getSampleStyleSheet()

elements.append(Paragraph("<b>INFORME DE VENTAS POR AÑO TIENDA 1</b>", styles['Title']))
elements.append(Spacer(1, 12))

resumen_pdf['total_anual'] = resumen_pdf['total_anual'].map('${:,.0f}'.format)
resumen_pdf['promedio_mensual'] = resumen_pdf['promedio_mensual'].map('${:,.0f}'.format)
resumen_pdf['porcentaje'] = resumen_pdf['porcentaje'].map('{:.2f}%'.format)
data_table = [list(resumen_pdf.columns)] + resumen_pdf.values.tolist()

table = Table(data_table, hAlign='CENTER')
table_style = TableStyle([
    ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor("#4F81BD")),
    ('TEXTCOLOR', (0, 0), (-1, 0), colors.white),
    ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
    ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
    ('FONTSIZE', (0, 0), (-1, -1), 10),
    ('GRID', (0, 0), (-1, -1), 0.5, colors.grey),
    ('BACKGROUND', (0, 1), (-1, -1), colors.whitesmoke),
    ('BACKGROUND', (0, -1), (-1, -1), colors.HexColor("#D9E1F2")),
    ('FONTNAME', (0, -1), (-1, -1), 'Helvetica-Bold')
])
table.setStyle(table_style)
elements.append(table)

# Generar PDF
doc.build(elements)
print(f"✅ Informe PDF generado en: {pdf_file}")


============ INFORME DE VENTAS POR AÑO TIENDA 1=============
╒═══════╤════════════════╤════════════════════╤══════════════╕
│ año   │ total_anual    │ promedio_mensual   │ porcentaje   │
╞═══════╪════════════════╪════════════════════╪══════════════╡
│ 2020  │ $368,933,200   │ $515,990           │ 32.06%       │
├───────┼────────────────┼────────────────────┼──────────────┤
│ 2021  │ $362,120,800   │ $490,015           │ 31.46%       │
├───────┼────────────────┼────────────────────┼──────────────┤
│ 2022  │ $316,565,600   │ $443,991           │ 27.51%       │
├───────┼────────────────┼────────────────────┼──────────────┤
│ 2023  │ $103,260,800   │ $537,817           │ 8.97%        │
├───────┼────────────────┼────────────────────┼──────────────┤
│ TOTAL │ $1,150,880,400 │ $496,953           │ 100.00%      │
╘═══════╧════════════════╧════════════════════╧══════════════╛
✅ Informe PDF generado en: /content/informe_ventas_tienda1.pdf


In [ ]:
import pandas as pd
from tabulate import tabulate
from reportlab.lib.pagesizes import A4
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet

df = pd.DataFrame(tienda2)
df['Fecha de Compra'] = pd.to_datetime(df['Fecha de Compra'], dayfirst=True)
df['año'] = df['Fecha de Compra'].dt.year

resumen = df.groupby('año').agg(
    total_anual=('Precio', 'sum'),
    promedio_mensual=('Precio', lambda x: x.sum() / len(x))
).reset_index()

total_general = resumen['total_anual'].sum()
resumen['porcentaje'] = (resumen['total_anual'] / total_general) * 100

fila_total = pd.DataFrame({
    'año': ['TOTAL'],
    'total_anual': [total_general],
    'promedio_mensual': [resumen['promedio_mensual'].mean()],
    'porcentaje': [100]
})
resumen = pd.concat([resumen, fila_total], ignore_index=True)

resumen_pdf = resumen.copy()

resumen['total_anual'] = resumen['total_anual'].map('${:,.0f}'.format)
resumen['promedio_mensual'] = resumen['promedio_mensual'].map('${:,.0f}'.format)
resumen['porcentaje'] = resumen['porcentaje'].map('{:.2f}%'.format)

print("\n" + "="*60)
print(" INFORME DE VENTAS POR AÑO TIENDA 2".center(60, "="))
print("="*60)
print(tabulate(resumen, headers="keys", tablefmt="fancy_grid", showindex=False))
print("="*60)

pdf_file = "/content/informe_ventas_tienda2.pdf"
doc = SimpleDocTemplate(pdf_file, pagesize=A4)
elements = []
styles = getSampleStyleSheet()

elements.append(Paragraph("<b>INFORME DE VENTAS POR AÑO TIENDA 2</b>", styles['Title']))
elements.append(Spacer(1, 12))

resumen_pdf['total_anual'] = resumen_pdf['total_anual'].map('${:,.0f}'.format)
resumen_pdf['promedio_mensual'] = resumen_pdf['promedio_mensual'].map('${:,.0f}'.format)
resumen_pdf['porcentaje'] = resumen_pdf['porcentaje'].map('{:.2f}%'.format)
data_table = [list(resumen_pdf.columns)] + resumen_pdf.values.tolist()

table = Table(data_table, hAlign='CENTER')
table_style = TableStyle([
    ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor("#4F81BD")),
    ('TEXTCOLOR', (0, 0), (-1, 0), colors.white),
    ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
    ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
    ('FONTSIZE', (0, 0), (-1, -1), 10),
    ('GRID', (0, 0), (-1, -1), 0.5, colors.grey),
    ('BACKGROUND', (0, 1), (-1, -1), colors.whitesmoke),
    ('BACKGROUND', (0, -1), (-1, -1), colors.HexColor("#D9E1F2")),
    ('FONTNAME', (0, -1), (-1, -1), 'Helvetica-Bold')
])
table.setStyle(table_style)
elements.append(table)

# Generar PDF
doc.build(elements)
print(f"✅ Informe PDF generado en: {pdf_file}")


============ INFORME DE VENTAS POR AÑO TIENDA 2=============
╒═══════╤════════════════╤════════════════════╤══════════════╕
│ año   │ total_anual    │ promedio_mensual   │ porcentaje   │
╞═══════╪════════════════╪════════════════════╪══════════════╡
│ 2020  │ $320,466,600   │ $457,809           │ 28.71%       │
├───────┼────────────────┼────────────────────┼──────────────┤
│ 2021  │ $351,215,500   │ $462,735           │ 31.46%       │
├───────┼────────────────┼────────────────────┼──────────────┤
│ 2022  │ $358,226,500   │ $505,970           │ 32.09%       │
├───────┼────────────────┼────────────────────┼──────────────┤
│ 2023  │ $86,434,900    │ $450,182           │ 7.74%        │
├───────┼────────────────┼────────────────────┼──────────────┤
│ TOTAL │ $1,116,343,500 │ $469,174           │ 100.00%      │
╘═══════╧════════════════╧════════════════════╧══════════════╛
✅ Informe PDF generado en: /content/informe_ventas_tienda2.pdf


In [ ]:
import pandas as pd
from tabulate import tabulate
from reportlab.lib.pagesizes import A4
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet

df = pd.DataFrame(tienda3)
df['Fecha de Compra'] = pd.to_datetime(df['Fecha de Compra'], dayfirst=True)
df['año'] = df['Fecha de Compra'].dt.year

resumen = df.groupby('año').agg(
    total_anual=('Precio', 'sum'),
    promedio_mensual=('Precio', lambda x: x.sum() / len(x))
).reset_index()

total_general = resumen['total_anual'].sum()
resumen['porcentaje'] = (resumen['total_anual'] / total_general) * 100

fila_total = pd.DataFrame({
    'año': ['TOTAL'],
    'total_anual': [total_general],
    'promedio_mensual': [resumen['promedio_mensual'].mean()],
    'porcentaje': [100]
})
resumen = pd.concat([resumen, fila_total], ignore_index=True)

resumen_pdf = resumen.copy()

resumen['total_anual'] = resumen['total_anual'].map('${:,.0f}'.format)
resumen['promedio_mensual'] = resumen['promedio_mensual'].map('${:,.0f}'.format)
resumen['porcentaje'] = resumen['porcentaje'].map('{:.2f}%'.format)

print("\n" + "="*60)
print(" INFORME DE VENTAS POR AÑO TIENDA 3".center(60, "="))
print("="*60)
print(tabulate(resumen, headers="keys", tablefmt="fancy_grid", showindex=False))
print("="*60)

pdf_file = "/content/informe_ventas_tienda3.pdf"
doc = SimpleDocTemplate(pdf_file, pagesize=A4)
elements = []
styles = getSampleStyleSheet()

elements.append(Paragraph("<b>INFORME DE VENTAS POR AÑO TIENDA 3</b>", styles['Title']))
elements.append(Spacer(1, 12))

resumen_pdf['total_anual'] = resumen_pdf['total_anual'].map('${:,.0f}'.format)
resumen_pdf['promedio_mensual'] = resumen_pdf['promedio_mensual'].map('${:,.0f}'.format)
resumen_pdf['porcentaje'] = resumen_pdf['porcentaje'].map('{:.2f}%'.format)
data_table = [list(resumen_pdf.columns)] + resumen_pdf.values.tolist()

table = Table(data_table, hAlign='CENTER')
table_style = TableStyle([
    ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor("#4F81BD")),
    ('TEXTCOLOR', (0, 0), (-1, 0), colors.white),
    ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
    ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
    ('FONTSIZE', (0, 0), (-1, -1), 10),
    ('GRID', (0, 0), (-1, -1), 0.5, colors.grey),
    ('BACKGROUND', (0, 1), (-1, -1), colors.whitesmoke),
    ('BACKGROUND', (0, -1), (-1, -1), colors.HexColor("#D9E1F2")),
    ('FONTNAME', (0, -1), (-1, -1), 'Helvetica-Bold')
])
table.setStyle(table_style)
elements.append(table)

# Generar PDF
doc.build(elements)
print(f"✅ Informe PDF generado en: {pdf_file}")


============ INFORME DE VENTAS POR AÑO TIENDA 3=============
╒═══════╤════════════════╤════════════════════╤══════════════╕
│ año   │ total_anual    │ promedio_mensual   │ porcentaje   │
╞═══════╪════════════════╪════════════════════╪══════════════╡
│ 2020  │ $321,707,500   │ $458,927           │ 29.30%       │
├───────┼────────────────┼────────────────────┼──────────────┤
│ 2021  │ $362,954,400   │ $490,479           │ 33.06%       │
├───────┼────────────────┼────────────────────┼──────────────┤
│ 2022  │ $350,438,300   │ $473,565           │ 31.92%       │
├───────┼────────────────┼────────────────────┼──────────────┤
│ 2023  │ $62,919,400    │ $353,480           │ 5.73%        │
├───────┼────────────────┼────────────────────┼──────────────┤
│ TOTAL │ $1,098,019,600 │ $444,113           │ 100.00%      │
╘═══════╧════════════════╧════════════════════╧══════════════╛
✅ Informe PDF generado en: /content/informe_ventas_tienda3.pdf


In [ ]:
import pandas as pd
from tabulate import tabulate
from reportlab.lib.pagesizes import A4
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet

df = pd.DataFrame(tienda4)
df['Fecha de Compra'] = pd.to_datetime(df['Fecha de Compra'], dayfirst=True)
df['año'] = df['Fecha de Compra'].dt.year

resumen = df.groupby('año').agg(
    total_anual=('Precio', 'sum'),
    promedio_mensual=('Precio', lambda x: x.sum() / len(x))
).reset_index()

total_general = resumen['total_anual'].sum()
resumen['porcentaje'] = (resumen['total_anual'] / total_general) * 100

fila_total = pd.DataFrame({
    'año': ['TOTAL'],
    'total_anual': [total_general],
    'promedio_mensual': [resumen['promedio_mensual'].mean()],
    'porcentaje': [100]
})
resumen = pd.concat([resumen, fila_total], ignore_index=True)

resumen_pdf = resumen.copy()

resumen['total_anual'] = resumen['total_anual'].map('${:,.0f}'.format)
resumen['promedio_mensual'] = resumen['promedio_mensual'].map('${:,.0f}'.format)
resumen['porcentaje'] = resumen['porcentaje'].map('{:.2f}%'.format)

print("\n" + "="*60)
print(" INFORME DE VENTAS POR AÑO TIENDA 4".center(60, "="))
print("="*60)
print(tabulate(resumen, headers="keys", tablefmt="fancy_grid", showindex=False))
print("="*60)

pdf_file = "/content/informe_ventas_tienda4.pdf"
doc = SimpleDocTemplate(pdf_file, pagesize=A4)
elements = []
styles = getSampleStyleSheet()

elements.append(Paragraph("<b>INFORME DE VENTAS POR AÑO TIENDA 4</b>", styles['Title']))
elements.append(Spacer(1, 12))

resumen_pdf['total_anual'] = resumen_pdf['total_anual'].map('${:,.0f}'.format)
resumen_pdf['promedio_mensual'] = resumen_pdf['promedio_mensual'].map('${:,.0f}'.format)
resumen_pdf['porcentaje'] = resumen_pdf['porcentaje'].map('{:.2f}%'.format)
data_table = [list(resumen_pdf.columns)] + resumen_pdf.values.tolist()

table = Table(data_table, hAlign='CENTER')
table_style = TableStyle([
    ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor("#4F81BD")),
    ('TEXTCOLOR', (0, 0), (-1, 0), colors.white),
    ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
    ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
    ('FONTSIZE', (0, 0), (-1, -1), 10),
    ('GRID', (0, 0), (-1, -1), 0.5, colors.grey),
    ('BACKGROUND', (0, 1), (-1, -1), colors.whitesmoke),
    ('BACKGROUND', (0, -1), (-1, -1), colors.HexColor("#D9E1F2")),
    ('FONTNAME', (0, -1), (-1, -1), 'Helvetica-Bold')
])
table.setStyle(table_style)
elements.append(table)

# Generar PDF
doc.build(elements)
print(f"✅ Informe PDF generado en: {pdf_file}")


============ INFORME DE VENTAS POR AÑO TIENDA 4=============
╒═══════╤════════════════╤════════════════════╤══════════════╕
│ año   │ total_anual    │ promedio_mensual   │ porcentaje   │
╞═══════╪════════════════╪════════════════════╪══════════════╡
│ 2020  │ $330,847,700   │ $444,091           │ 31.86%       │
├───────┼────────────────┼────────────────────┼──────────────┤
│ 2021  │ $347,823,600   │ $467,505           │ 33.50%       │
├───────┼────────────────┼────────────────────┼──────────────┤
│ 2022  │ $302,221,500   │ $424,468           │ 29.11%       │
├───────┼────────────────┼────────────────────┼──────────────┤
│ 2023  │ $57,482,900    │ $366,133           │ 5.54%        │
├───────┼────────────────┼────────────────────┼──────────────┤
│ TOTAL │ $1,038,375,700 │ $425,549           │ 100.00%      │
╘═══════╧════════════════╧════════════════════╧══════════════╛
✅ Informe PDF generado en: /content/informe_ventas_tienda4.pdf


1.3 Informe comparativo por tiendas

In [ ]:
import pandas as pd
from tabulate import tabulate
from reportlab.lib.pagesizes import A4
from reportlab.lib import colors
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet


df_tiendaA = pd.DataFrame(tienda1)
df_tiendaB = pd.DataFrame(tienda2)
df_tiendaC = pd.DataFrame(tienda3)
df_tiendaD = pd.DataFrame(tienda4)

df_tiendaA['Tienda'] = 'Tienda A'
df_tiendaB['Tienda'] = 'Tienda B'
df_tiendaC['Tienda'] = 'Tienda C'
df_tiendaD['Tienda'] = 'Tienda D'

df = pd.concat([df_tiendaA, df_tiendaB, df_tiendaC, df_tiendaD], ignore_index=True)

df['Fecha de Compra'] = pd.to_datetime(df['Fecha de Compra'], dayfirst=True)

df['Año'] = df['Fecha de Compra'].dt.year

tiendas = ['Tienda A', 'Tienda B', 'Tienda C', 'Tienda D']
pivot_total = df.groupby(['Año', 'Tienda'])['Precio'].sum().reset_index()
pivot_total = pivot_total.pivot(index='Año', columns='Tienda', values='Precio').reset_index()
pivot_total['Total_Año'] = pivot_total[tiendas].sum(axis=1)

porcentajes = pivot_total.copy()
for t in tiendas:
    porcentajes[t] = (pivot_total[t] / pivot_total['Total_Año']) * 100
porcentajes['Total_Año'] = 100

tabla_final = []
for i in range(len(pivot_total)):
    fila = [pivot_total.loc[i, 'Año']]
    for t in tiendas + ['Total_Año']:
        monto = pivot_total.loc[i, t]
        porc = porcentajes.loc[i, t]
        fila.append(f"${monto:,.0f} ({porc:.2f}%)")
    tabla_final.append(fila)

fila_total = ["Total"]
for t in tiendas + ['Total_Año']:
    monto = pivot_total[t].sum()
    porc = (monto / pivot_total['Total_Año'].sum() * 100)
    fila_total.append(f"${monto:,.0f} ({porc:.2f}%)")
tabla_final.append(fila_total)

headers = ["Año"] + tiendas + ["Total_Año"]
print("\n" + "="*100)
print(" Comparativo de Ventas por Año, Tienda y Participación (%) ".center(100, "="))
print("="*100)
print(tabulate(tabla_final, headers=headers, tablefmt="fancy_grid", stralign="center"))
print("="*100)

def exportar_pdf(nombre_archivo, titulo, tabla, headers):
    doc = SimpleDocTemplate(nombre_archivo, pagesize=A4)
    estilos = getSampleStyleSheet()
    contenido = []


    contenido.append(Paragraph(titulo, estilos['Title']))
    contenido.append(Spacer(1, 12))


    data_pdf = [headers] + tabla
    tabla_pdf = Table(data_pdf, repeatRows=1)
    tabla_pdf.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor("#4F81BD")),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('FONTSIZE', (0, 0), (-1, 0), 10),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 8),
        ('GRID', (0, 0), (-1, -1), 0.25, colors.black),
        ('FONTNAME', (0, 1), (-1, -1), 'Helvetica'),
        ('FONTSIZE', (0, 1), (-1, -1), 9),
    ]))

    contenido.append(tabla_pdf)
    doc.build(contenido)

exportar_pdf(
    "comparativo_ventas_4_tiendas.pdf",
    "Comparativo de Ventas por Año, Tienda y Participación (%)",
    tabla_final,
    headers
)

print("\nPDF generado: comparativo_ventas_4_tiendas.pdf ✅")


==================== Comparativo de Ventas por Año, Tienda y Participación (%) =====================
╒═══════╤═════════════════════════╤═════════════════════════╤═════════════════════════╤═════════════════════════╤══════════════════════════╕
│  Año  │        Tienda A         │        Tienda B         │        Tienda C         │        Tienda D         │        Total_Año         │
╞═══════╪═════════════════════════╪═════════════════════════╪═════════════════════════╪═════════════════════════╪══════════════════════════╡
│ 2020  │  $368,933,200 (27.49%)  │  $320,466,600 (23.88%)  │  $321,707,500 (23.97%)  │  $330,847,700 (24.65%)  │ $1,341,955,000 (100.00%) │
├───────┼─────────────────────────┼─────────────────────────┼─────────────────────────┼─────────────────────────┼──────────────────────────┤
│ 2021  │  $362,120,800 (25.43%)  │  $351,215,500 (24.66%)  │  $362,954,400 (25.49%)  │  $347,823,600 (24.42%)  │ $1,424,114,300 (100.00%) │
├───────┼─────────────────────────┼─────────────────

# 2. Ventas por categoría

In [ ]:
dftienda1=pd.DataFrame(tienda1)
ventas_cat_tienda1=(dftienda1.groupby('Categoría del Producto', as_index=False )['Precio'].sum().sort_values(by ='Precio', ascending=False))
ventas_cat_tienda1['Precio']=ventas_cat_tienda1['Precio'].apply(lambda x:f"${x:,.0f}")
print("\nVentas por categoría Tienda1")
print(ventas_cat_tienda1.to_string(index=False))


Ventas por categoría Tienda1
 Categoría del Producto       Precio
           Electrónicos $429,493,500
      Electrodomésticos $363,685,200
                Muebles $187,633,700
 Instrumentos musicales  $91,299,000
   Deportes y diversión  $39,290,000
               Juguetes  $17,995,700
Artículos para el hogar  $12,698,400
                 Libros   $8,784,900


In [ ]:
dftienda2=pd.DataFrame(tienda2)
ventas_cat_tienda2=(dftienda2.groupby('Categoría del Producto', as_index=False )['Precio'].sum().sort_values(by ='Precio', ascending=False))
ventas_cat_tienda2['Precio']=ventas_cat_tienda2['Precio'].apply(lambda x:f"${x:,.0f}")
print("\nVentas por categoría Tienda2")
print(ventas_cat_tienda2.to_string(index=False))


Ventas por categoría Tienda2
 Categoría del Producto       Precio
           Electrónicos $410,831,100
      Electrodomésticos $348,567,800
                Muebles $176,426,300
 Instrumentos musicales $104,990,300
   Deportes y diversión  $34,744,500
               Juguetes  $15,945,400
Artículos para el hogar  $14,746,900
                 Libros  $10,091,200


In [ ]:
dftienda3=pd.DataFrame(tienda3)
ventas_cat_tienda3=(dftienda3.groupby('Categoría del Producto', as_index=False )['Precio'].sum().sort_values(by ='Precio', ascending=False))
ventas_cat_tienda3['Precio']=ventas_cat_tienda3['Precio'].apply(lambda x:f"${x:,.0f}")
print("\nVentas por categoría Tienda3")
print(ventas_cat_tienda3.to_string(index=False))


Ventas por categoría Tienda3
 Categoría del Producto       Precio
           Electrónicos $410,775,800
      Electrodomésticos $329,237,900
                Muebles $201,072,100
 Instrumentos musicales  $77,380,900
   Deportes y diversión  $35,593,100
               Juguetes  $19,401,100
Artículos para el hogar  $15,060,000
                 Libros   $9,498,700


In [ ]:
dftienda4=pd.DataFrame(tienda4)
ventas_cat_tienda4=(dftienda4.groupby('Categoría del Producto', as_index=False )['Precio'].sum().sort_values(by ='Precio', ascending=False))
ventas_cat_tienda4['Precio']=ventas_cat_tienda4['Precio'].apply(lambda x:f"${x:,.0f}")
print("\nVentas por categoría Tienda4")
print(ventas_cat_tienda4.to_string(index=False))


Ventas por categoría Tienda4
 Categoría del Producto       Precio
           Electrónicos $409,476,100
      Electrodomésticos $283,260,200
                Muebles $192,528,900
 Instrumentos musicales  $75,102,400
   Deportes y diversión  $33,350,100
               Juguetes  $20,262,200
Artículos para el hogar  $15,074,500
                 Libros   $9,321,300


# 3. Calificación promedio de la tienda


In [ ]:

resultado=(tienda1.groupby('Calificación').size().pipe(lambda s: s.to_frame('Cantidad').assign(Proporcion=lambda x: (x['Cantidad'] / len(tienda1)).map("{:.2%}".format))).reset_index().sort_values(by='Cantidad', ascending=False))
print(resultado)

   Calificación  Cantidad Proporcion
4             5      1292     54.77%
3             4       444     18.82%
0             1       319     13.52%
2             3       218      9.24%
1             2        86      3.65%


In [ ]:
resultado=(tienda2.groupby('Calificación').size().pipe(lambda s: s.to_frame('Cantidad').assign(Proporcion=lambda x: (x['Cantidad'] / len(tienda2)).map("{:.2%}".format))).reset_index().sort_values(by='Cantidad', ascending=False))
print(resultado)

   Calificación  Cantidad Proporcion
4             5      1302     55.19%
3             4       482     20.43%
0             1       273     11.57%
2             3       209      8.86%
1             2        93      3.94%


In [ ]:
resultado=(tienda3.groupby('Calificación').size().pipe(lambda s: s.to_frame('Cantidad').assign(Proporcion=lambda x: (x['Cantidad'] / len(tienda3)).map("{:.2%}".format))).reset_index().sort_values(by='Cantidad', ascending=False))
print(resultado)

   Calificación  Cantidad Proporcion
4             5      1356     57.48%
3             4       434     18.40%
0             1       297     12.59%
2             3       193      8.18%
1             2        79      3.35%


In [ ]:
resultado=(tienda4.groupby('Calificación').size().pipe(lambda s: s.to_frame('Cantidad').assign(Proporcion=lambda x: (x['Cantidad'] / len(tienda4)).map("{:.2%}".format))).reset_index().sort_values(by='Cantidad', ascending=False))
print(resultado)

   Calificación  Cantidad Proporcion
4             5      1300     55.13%
3             4       453     19.21%
0             1       310     13.15%
2             3       210      8.91%
1             2        85      3.60%


# 4. Productos más y menos vendidos

In [ ]:
df_agrupado = tienda1.groupby('Producto', as_index=False)['Precio'].sum()
df_ordenado = df_agrupado.sort_values(by='Precio', ascending=True)
n = min(5, len(df_ordenado))
menos_vendidos = df_ordenado.head(n).copy()
menos_vendidos["ranking"] = range(1, len(menos_vendidos) + 1)
mas_vendidos = df_ordenado.tail(n).sort_values(by='Precio', ascending=False).copy()
mas_vendidos["ranking"] = range(1, len(mas_vendidos) + 1)
def mostrar_tabla(tienda1, titulo):
    print(f"\n{titulo.center(50, '=')}")
    print(tienda1.to_string(index=False,
                       formatters={
                           'Precio': lambda x: f"{x:,.0f}".replace(",", "."),
                           'ranking': lambda x: f"#{x}"
                       }))
mostrar_tabla(mas_vendidos, "PRODUCTOS MÁS VENDIDOS TIENDA 1")
mostrar_tabla(menos_vendidos, "PRODUCTOS MENOS VENDIDOS TIENDA 1")


=========PRODUCTOS MÁS VENDIDOS TIENDA 1==========
        Producto      Precio ranking
   TV LED UHD 4K 142.150.200      #1
    Refrigerador 112.143.300      #2
       Iphone 15  97.902.200      #3
        Smart TV  67.452.100      #4
Lavadora de ropa  65.783.600      #5

========PRODUCTOS MENOS VENDIDOS TIENDA 1=========
          Producto    Precio ranking
Cuerda para saltar   652.800      #1
    Dinosaurio Rex   688.900      #2
   Cubo mágico 8x8   696.900      #3
      Set de vasos 1.359.100      #4
 Ajedrez de madera 1.430.600      #5


In [ ]:
df_agrupado = tienda2.groupby('Producto', as_index=False)['Precio'].sum()
df_ordenado = df_agrupado.sort_values(by='Precio', ascending=True)
n = min(5, len(df_ordenado))
menos_vendidos = df_ordenado.head(n).copy()
menos_vendidos["ranking"] = range(1, len(menos_vendidos) + 1)
mas_vendidos = df_ordenado.tail(n).sort_values(by='Precio', ascending=False).copy()
mas_vendidos["ranking"] = range(1, len(mas_vendidos) + 1)
def mostrar_tabla(tienda2, titulo):
    print(f"\n{titulo.center(50, '=')}")
    print(tienda2.to_string(index=False,
                       formatters={
                           'Precio': lambda x: f"{x:,.0f}".replace(",", "."),
                           'ranking': lambda x: f"#{x}"
                       }))
mostrar_tabla(mas_vendidos, "PRODUCTOS MÁS VENDIDOS TIENDA 2")
mostrar_tabla(menos_vendidos, "PRODUCTOS MENOS VENDIDOS TIENDA 2")


=========PRODUCTOS MÁS VENDIDOS TIENDA 2==========
        Producto      Precio ranking
       Iphone 15 113.225.800      #1
   TV LED UHD 4K 103.375.000      #2
    Refrigerador  97.632.500      #3
Lavadora de ropa  66.762.300      #4
        Smart TV  61.142.000      #5

========PRODUCTOS MENOS VENDIDOS TIENDA 2=========
          Producto    Precio ranking
   Cubo mágico 8x8   643.600      #1
    Dinosaurio Rex   770.300      #2
Cuerda para saltar   856.900      #3
 Ajedrez de madera 1.287.700      #4
      Set de vasos 1.542.600      #5


In [ ]:
df_agrupado = tienda3.groupby('Producto', as_index=False)['Precio'].sum()
df_ordenado = df_agrupado.sort_values(by='Precio', ascending=True)
n = min(5, len(df_ordenado))
menos_vendidos = df_ordenado.head(n).copy()
menos_vendidos["ranking"] = range(1, len(menos_vendidos) + 1)
mas_vendidos = df_ordenado.tail(n).sort_values(by='Precio', ascending=False).copy()
mas_vendidos["ranking"] = range(1, len(mas_vendidos) + 1)
def mostrar_tabla(tienda3, titulo):
    print(f"\n{titulo.center(50, '=')}")
    print(tienda3.to_string(index=False,
                       formatters={
                           'Precio': lambda x: f"{x:,.0f}".replace(",", "."),
                           'ranking': lambda x: f"#{x}"
                       }))
mostrar_tabla(mas_vendidos, "PRODUCTOS MÁS VENDIDOS TIENDA 3")
mostrar_tabla(menos_vendidos, "PRODUCTOS MENOS VENDIDOS TIENDA 3")


=========PRODUCTOS MÁS VENDIDOS TIENDA 3==========
     Producto     Precio ranking
 Refrigerador 99.889.700      #1
TV LED UHD 4K 96.669.100      #2
    Iphone 15 93.225.700      #3
     Smart TV 78.837.300      #4
 Lavavajillas 64.364.800      #5

========PRODUCTOS MENOS VENDIDOS TIENDA 3=========
               Producto    Precio ranking
        Cubo mágico 8x8   640.400      #1
         Dinosaurio Rex   795.600      #2
     Cuerda para saltar   853.800      #3
           Set de vasos 1.210.700      #4
Bloques de construcción 1.237.200      #5


In [ ]:
df_agrupado = tienda4.groupby('Producto', as_index=False)['Precio'].sum()
df_ordenado = df_agrupado.sort_values(by='Precio', ascending=True)
n = min(5, len(df_ordenado))
menos_vendidos = df_ordenado.head(n).copy()
menos_vendidos["ranking"] = range(1, len(menos_vendidos) + 1)
mas_vendidos = df_ordenado.tail(n).sort_values(by='Precio', ascending=False).copy()
mas_vendidos["ranking"] = range(1, len(mas_vendidos) + 1)
def mostrar_tabla(tienda4, titulo):
    print(f"\n{titulo.center(50, '=')}")
    print(tienda4.to_string(index=False,
                       formatters={
                           'Precio': lambda x: f"{x:,.0f}".replace(",", "."),
                           'ranking': lambda x: f"#{x}"
                       }))
mostrar_tabla(mas_vendidos, "PRODUCTOS MÁS VENDIDOS TIENDA 4")
mostrar_tabla(menos_vendidos, "PRODUCTOS MENOS VENDIDOS TIENDA 4")


=========PRODUCTOS MÁS VENDIDOS TIENDA 4==========
     Producto     Precio ranking
    Iphone 15 96.697.500      #1
TV LED UHD 4K 90.295.300      #2
     Smart TV 82.790.400      #3
 Refrigerador 75.271.900      #4
  Tablet ABXY 56.723.100      #5

========PRODUCTOS MENOS VENDIDOS TIENDA 4=========
          Producto    Precio ranking
Cuerda para saltar   704.800      #1
   Cubo mágico 8x8   748.400      #2
    Dinosaurio Rex   858.100      #3
 Ajedrez de madera 1.075.500      #4
      Set de vasos 1.471.800      #5


# 5. Envío promedio por tienda

In [ ]:
df = pd.DataFrame(tienda1)
df['Fecha de Compra'] = pd.to_datetime(df['Fecha de Compra'], dayfirst=True)
grouped = df.groupby(df['Fecha de Compra'].dt.year)['Costo de envío']
total_por_anio = grouped.sum()
promedio_por_anio = grouped.mean()
total_general = total_por_anio.sum()
porcentaje = (total_por_anio / total_general) * 100
resumen = pd.DataFrame({
    'Año': total_por_anio.index,
    'Total envíos': total_por_anio.values,
    'Promedio envíos': promedio_por_anio.values,
    'Porcentaje': porcentaje.values
})
resumen['Total envíos'] = resumen['Total envíos'].apply(lambda x: f"${x:,.0f}")
resumen['Promedio envíos'] = resumen['Promedio envíos'].apply(lambda x: f"${x:,.2f}")
resumen['Porcentaje'] = resumen['Porcentaje'].apply(lambda x: f"{x:.2f}%")
print('======== Costo promedio de Envío Tienda 1 ========')
print(resumen)

======== Costo promedio de Envío Tienda 1 ========
    Año Total envíos Promedio envíos Porcentaje
0  2020  $19,746,200      $27,617.06     32.17%
1  2021  $19,219,700      $26,007.71     31.31%
2  2022  $16,862,300      $23,649.79     27.47%
3  2023   $5,549,700      $28,904.69      9.04%


In [ ]:
df = pd.DataFrame(tienda2)
df['Fecha de Compra'] = pd.to_datetime(df['Fecha de Compra'], dayfirst=True)
grouped = df.groupby(df['Fecha de Compra'].dt.year)['Costo de envío']
total_por_anio = grouped.sum()
promedio_por_anio = grouped.mean()
total_general = total_por_anio.sum()
porcentaje = (total_por_anio / total_general) * 100
resumen = pd.DataFrame({
    'Año': total_por_anio.index,
    'Total envíos': total_por_anio.values,
    'Promedio envíos': promedio_por_anio.values,
    'Porcentaje': porcentaje.values
})
resumen['Total envíos'] = resumen['Total envíos'].apply(lambda x: f"${x:,.0f}")
resumen['Promedio envíos'] = resumen['Promedio envíos'].apply(lambda x: f"${x:,.2f}")
resumen['Porcentaje'] = resumen['Porcentaje'].apply(lambda x: f"{x:.2f}%")
print('======== Costo promedio de Envío Tienda 2 ========')
print(resumen)

======== Costo promedio de Envío Tienda 2 ========
    Año Total envíos Promedio envíos Porcentaje
0  2020  $17,167,500      $24,525.00     28.86%
1  2021  $18,618,500      $24,530.30     31.30%
2  2022  $19,094,700      $26,969.92     32.10%
3  2023   $4,604,400      $23,981.25      7.74%


In [ ]:
df = pd.DataFrame(tienda3)
df['Fecha de Compra'] = pd.to_datetime(df['Fecha de Compra'], dayfirst=True)
grouped = df.groupby(df['Fecha de Compra'].dt.year)['Costo de envío']
total_por_anio = grouped.sum()
promedio_por_anio = grouped.mean()
total_general = total_por_anio.sum()
porcentaje = (total_por_anio / total_general) * 100
resumen = pd.DataFrame({
    'Año': total_por_anio.index,
    'Total envíos': total_por_anio.values,
    'Promedio envíos': promedio_por_anio.values,
    'Porcentaje': porcentaje.values
})
resumen['Total envíos'] = resumen['Total envíos'].apply(lambda x: f"${x:,.0f}")
resumen['Promedio envíos'] = resumen['Promedio envíos'].apply(lambda x: f"${x:,.2f}")
resumen['Porcentaje'] = resumen['Porcentaje'].apply(lambda x: f"{x:.2f}%")
print('======== Costo promedio de Envío Tienda 3 ========')
print(resumen)

======== Costo promedio de Envío Tienda 3 ========
    Año Total envíos Promedio envíos Porcentaje
0  2020  $17,067,900      $24,347.93     29.17%
1  2021  $19,384,800      $26,195.68     33.13%
2  2022  $18,728,300      $25,308.51     32.01%
3  2023   $3,335,600      $18,739.33      5.70%


In [ ]:
df = pd.DataFrame(tienda4)
df['Fecha de Compra'] = pd.to_datetime(df['Fecha de Compra'], dayfirst=True)
grouped = df.groupby(df['Fecha de Compra'].dt.year)['Costo de envío']
total_por_anio = grouped.sum()
promedio_por_anio = grouped.mean()
total_general = total_por_anio.sum()
porcentaje = (total_por_anio / total_general) * 100
resumen = pd.DataFrame({
    'Año': total_por_anio.index,
    'Total envíos': total_por_anio.values,
    'Promedio envíos': promedio_por_anio.values,
    'Porcentaje': porcentaje.values
})
resumen['Total envíos'] = resumen['Total envíos'].apply(lambda x: f"${x:,.0f}")
resumen['Promedio envíos'] = resumen['Promedio envíos'].apply(lambda x: f"${x:,.2f}")
resumen['Porcentaje'] = resumen['Porcentaje'].apply(lambda x: f"{x:.2f}%")
print('======== Costo promedio de Envío Tienda 4 ========')
print(resumen)

======== Costo promedio de Envío Tienda 4 ========
    Año Total envíos Promedio envíos Porcentaje
0  2020  $17,528,600      $23,528.32     31.69%
1  2021  $18,598,200      $24,997.58     33.62%
2  2022  $16,156,400      $22,691.57     29.21%
3  2023   $3,034,200      $19,326.11      5.49%


# 6. Informe final

Estimados señores, de acuerdo a los analisis de datos efectuados e informes generados, además del cuadro comparativo,  no se ven diferencias significativas de las ventas entre las tiendas.
Si la decisión de cerrar una tienda persiste la que muestra una baja pero no considerable es la tienda número 4.
Esperando que está información sea de ayuda en la toma de una decisión, me despido.

Saludos cordiales

Ignacio Gorigoitia G.

Analista de Datos